<a href="https://colab.research.google.com/github/Kshitij0412/ML-projects/blob/main/Copy_of_fcc_sms_text_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf
import pandas as pd
from tensorflow import keras
!pip install tensorflow-datasets
import tensorflow_datasets as tfds
import numpy as np
import matplotlib.pyplot as plt

print(tf.__version__)

2.18.0


In [2]:
# get data files
!wget https://cdn.freecodecamp.org/project-data/sms/train-data.tsv
!wget https://cdn.freecodecamp.org/project-data/sms/valid-data.tsv
train_file_path= "train-data.tsv"
test_file_path = "valid-data.tsv"

--2025-03-26 07:38:40--  https://cdn.freecodecamp.org/project-data/sms/train-data.tsv
Resolving cdn.freecodecamp.org (cdn.freecodecamp.org)... 104.26.2.33, 172.67.70.149, 104.26.3.33, ...
Connecting to cdn.freecodecamp.org (cdn.freecodecamp.org)|104.26.2.33|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 358233 (350K) [text/tab-separated-values]
Saving to: ‘train-data.tsv’

train-data.tsv      100%[===================>] 349.84K  --.-KB/s    in 0.02s   

2025-03-26 07:38:40 (21.1 MB/s) - ‘train-data.tsv’ saved [358233/358233]

--2025-03-26 07:38:41--  https://cdn.freecodecamp.org/project-data/sms/valid-data.tsv
Resolving cdn.freecodecamp.org (cdn.freecodecamp.org)... 104.26.2.33, 172.67.70.149, 104.26.3.33, ...
Connecting to cdn.freecodecamp.org (cdn.freecodecamp.org)|104.26.2.33|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 118774 (116K) [text/tab-separated-values]
Saving to: ‘valid-data.tsv’

valid-data.tsv      100%[==============

In [3]:
df_train = pd.read_table(train_file_path, header=0, names=['type', 'msg'], usecols=['type', 'msg'])
df_test = pd.read_table(test_file_path, header=0, names=['type', 'msg'], usecols=['type', 'msg'])


In [4]:
df_train['type'] = df_train['type'].map({"ham": 0, "spam": 1})
df_test['type'] = df_test['type'].map({"ham": 0, "spam": 1})

In [5]:
tokenizer = tfds.deprecated.text.Tokenizer()
vocabulary_set = set()

In [6]:
for msg in df_train['msg'].values:
    some_tokens = tokenizer.tokenize(msg)
    vocabulary_set.update(some_tokens)

In [7]:
for msg in df_test['msg'].values:
    some_tokens = tokenizer.tokenize(msg)
    vocabulary_set.update(some_tokens)

In [8]:
vocab_size = len(vocabulary_set)
encoder = tfds.deprecated.text.TokenTextEncoder(vocabulary_set)

In [9]:
def encode(text, label):
    encoded_text = encoder.encode(text.numpy())
    return encoded_text, label

In [10]:
def encode_map_fn(text, label):
    encoded_text, label = tf.py_function(encode, inp=[text, label], Tout=(tf.int64, tf.int64))
    encoded_text.set_shape([None])
    label.set_shape([])
    return encoded_text, label

In [11]:
train_data = tf.data.Dataset.from_tensor_slices((df_train['msg'].values, df_train['type'].values))
test_data = tf.data.Dataset.from_tensor_slices((df_test['msg'].values, df_test['type'].values))

In [12]:
train_data_encoded = train_data.map(encode_map_fn)
test_data_encoded = test_data.map(encode_map_fn)

In [13]:
BUFFER_SIZE = 1000
BATCH_SIZE = 32
train_dataset = train_data_encoded.shuffle(BUFFER_SIZE).padded_batch(BATCH_SIZE)
test_dataset = test_data_encoded.padded_batch(BATCH_SIZE)

In [14]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(encoder.vocab_size, 32),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32)),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(1)
])

In [15]:
model.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=True), optimizer='adam', metrics=['accuracy'])

In [16]:
history = model.fit(train_dataset, epochs=10, validation_data=test_dataset, validation_steps=30)

Epoch 1/10
131/131 ━━━━━━━━━━━━━━━━━━━━ 15s 69ms/step - accuracy: 0.8755 - loss: 0.3646 - val_accuracy: 0.9698 - val_loss: 0.0917
Epoch 2/10
131/131 ━━━━━━━━━━━━━━━━━━━━ 10s 70ms/step - accuracy: 0.9845 - loss: 0.0542 - val_accuracy: 0.9854 - val_loss: 0.0459
Epoch 3/10
131/131 ━━━━━━━━━━━━━━━━━━━━ 10s 68ms/step - accuracy: 0.9953 - loss: 0.0180 - val_accuracy: 0.9896 - val_loss: 0.0528
Epoch 4/10
131/131 ━━━━━━━━━━━━━━━━━━━━ 10s 75ms/step - accuracy: 0.9981 - loss: 0.0066 - val_accuracy: 0.9865 - val_loss: 0.0693
Epoch 5/10
131/131 ━━━━━━━━━━━━━━━━━━━━ 10s 75ms/step - accuracy: 0.9982 - loss: 0.0036 - val_accuracy: 0.9885 - val_loss: 0.0611
Epoch 6/10
131/131 ━━━━━━━━━━━━━━━━━━━━ 10s 75ms/step - accuracy: 0.9998 - loss: 0.0014 - val_accuracy: 0.9906 - val_loss: 0.0612
Epoch 7/10
131/131 ━━━━━━━━━━━━━━━━━━━━ 21s 77ms/step - accuracy: 0.9999 - loss: 4.4941e-04 - val_accuracy: 0.9896 - val_loss: 0.0639
Epoch 8/10
131/131 ━━━━━━━━━━━━━━━━━━━━ 20s 75ms/step - accuracy: 0.9999 - loss: 3.415

In [17]:
test_loss, test_acc = model.evaluate(test_dataset)
print('Test Loss: {}'.format(test_loss))
print('Test Accuracy: {}'.format(test_acc))

44/44 ━━━━━━━━━━━━━━━━━━━━ 1s 20ms/step - accuracy: 0.9891 - loss: 0.0778
Test Loss: 0.07740750163793564
Test Accuracy: 0.9884974956512451


In [18]:
def predict_message(pred_text):
    pred_text_encoded = encoder.encode(pred_text)
    pred_text_encoded = tf.cast(pred_text_encoded, tf.float32)
    prediction = model.predict(tf.expand_dims(pred_text_encoded, 0)).tolist()
    if prediction[0][0] < 0.5:
        return pred_text, 'ham'
    else:
        return pred_text, 'spam'

In [19]:
# Run this cell to test your function and model. Do not modify contents.
def test_predictions():
  test_messages = ["how are you doing today",
                   "sale today! to stop texts call 98912460324",
                   "i dont want to go. can we try it a different day? available sat",
                   "our new mobile video service is live. just install on your phone to start watching.",
                   "you have won £1000 cash! call to claim your prize.",
                   "i'll bring it tomorrow. don't forget the milk.",
                   "wow, is your arm alright. that happened to me one time too"
                  ]

  test_answers = ["ham", "spam", "ham", "spam", "spam", "ham", "ham"]
  passed = True

  for msg, ans in zip(test_messages, test_answers):
    prediction = predict_message(msg)
    if prediction[1] != ans:
      passed = False

  if passed:
    print("You passed the challenge. Great job!")
  else:
    print("You haven't passed yet. Keep trying.")

test_predictions()


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 432ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 431ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
You passed the challenge. Great job!
